In [1]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("HQXFDWYCOW3UP3IG6YSV7UNN-UMGVUDNEP7S5VQDX")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 7.0 MB/s 
     |████████████████████████████████| 51 kB 209 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=e48e99ee46915dcfc9833683265b449ca9bc42a3bde292c1eccd3746b6550fbd
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as pl
import cv2 as cv

In [3]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/model.h5')
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 14, 14, 64)       

In [22]:
from os import pread
path = "/content/drive/MyDrive/valid_dataset/Tomato___Septoria_leaf_spot/021f9747-f308-4694-859b-10f541a0e63c___JR_Sept.L.S 2506.JPG"

def single_prediction(path):
    img=cv.imread(path)
    img=cv.cvtColor(img,cv.COLOR_BGR2RGB)
    img=cv.resize(img,(64,64))
    return img.reshape(-1,64,64,3)

types=['BACTERIAL SPOT','EARLY BLIGHT','HEALTHY','LATE BLIGHT','LEAF MOLD','SEPTORIA LEAF SPOT','SPIDER MITE','TARGET SPOT','MOSAIC VIRUS','YELLOW LEAF CURL VIRUS']
description=['Bacterial spot of tomato is a potentially devastating disease that, in severe cases, can lead to unmarketable fruit and even plant death. Bacterial spot can occur wherever tomatoes are grown, but is found most frequently in warm, wet climates, as well as in greenhouses.',
             'Early blight is a common tomato disease caused by the fungus Alternaria solani. It can affect almost all parts of the tomato plants, including the leaves, stems, and fruits. The plants may not die, but they will be weakened and will set fewer tomatoes than normal.',
             'this is 3 disease description',
             'Late blight is a potentially devastating disease of tomato and potato, infecting leaves, stems, tomato fruit, and potato tubers. The disease spreads quickly in fields and can result in total crop failure if untreated. Late blight does not occur every year in Minnesota.',
             'Tomato leaf mold is a fungal disease that can develop when there are extended periods of leaf wetness and the relative humidity is high (greater than 85 percent). Due to this moisture requirement, the disease is seen primarily in hoophouses and greenhouses',
             'Septoria leaf spot starts on lower leaves as small, circular gray lesions (spots) with dark borders. Fungal lesions enlarge, coalesce, and cause leaves to yellow and die. Lesions usually appear when the first fruit begins to form. Tiny black pycnidia (fungal fruiting bodies) can be seen in the lesions.',
             'The tomato red spider mite can be found on both sides of leaves but it prefers the undersides near the leaf veins. Feeding causes leaves to become yellowish white and mottled. Tomato red spider mite produces webbing, especially on the undersides of leaves. In high infestations, dense webbing can mummify plants.',
             'Also known as early blight, target spot of tomato is a fungal disease that attacks a diverse assortment of plants, including papaya, peppers, snap beans, potatoes, cantaloupe, and squash as well as passionflower and certain ornamentals.',
             'Tomato mosaic virus (ToMV) is a member of family tobamoviridae and belongs to the genus tobamovirus, which is a plant pathogenic virus.The tomato crop is highly susceptible to the Tomato mosaic virus (ToMV). The symptoms vary from tiles, wrinkle, reduction and curvature of leaflets, and irregular ripening of fruits. This disease requires attention because of its easy dissemination by contact, cultural practices, or contaminated seed.',
             'Tomato yellow leaf curl virus is a species in the genus Begomovirus and family Geminiviridae. Tomato yellow leaf curl virus (TYLCV) infection induces severe symptoms on tomato plants and causes serious yield losses worldwide. TYLCV is persistently transmitted by the sweetpotato whitefly, Bemisia tabaci (Gennadius).']

def predictor(path):
  image = single_prediction(path)
  prediction = new_model.predict(image)
  r=np.argmax(prediction,axis=1)
  pred = types[r[0]]
  return pred

def details(path):
  image = single_prediction(path)
  prediction = new_model.predict(image)
  r=np.argmax(prediction,axis=1)
  descrp =description[r[0]]
  return descrp


In [23]:
predictor("/content/drive/MyDrive/valid_dataset/Tomato___Septoria_leaf_spot/021f9747-f308-4694-859b-10f541a0e63c___JR_Sept.L.S 2506.JPG")

'SEPTORIA LEAF SPOT'

In [24]:
details("/content/drive/MyDrive/valid_dataset/Tomato___Septoria_leaf_spot/021f9747-f308-4694-859b-10f541a0e63c___JR_Sept.L.S 2506.JPG")

'Septoria leaf spot starts on lower leaves as small, circular gray lesions (spots) with dark borders. Fungal lesions enlarge, coalesce, and cause leaves to yellow and die. Lesions usually appear when the first fruit begins to form. Tiny black pycnidia (fungal fruiting bodies) can be seen in the lesions.'

In [26]:
import anvil.media

@anvil.server.callable
def detect(file):
  with anvil.media.TempFile(file) as file_name:
    prediction = predictor(file_name)
    description = details(file_name)
  return prediction
  
@anvil.server.callable
def descriptor(file):
  with anvil.media.TempFile(file) as file_name:
    description = details(file_name)
  return description

In [27]:
anvil.server.wait_forever()

KeyboardInterrupt: ignored